# Introduction to TileMatrixSets

The goal of this notebook is to give a quick introduction to the new TMS option in rio-tiler.

# Requirements

To be able to run this notebook you'll need the following requirements:
- rio-tiler~= 2.0
- ipyleaflet

In [1]:
# !pip install rio-tiler
# !pip install ipyleaflet

In [2]:
import morecantile
import requests
from ipyleaflet import (
    Map,
    basemaps,
    basemap_to_tiles,
    TileLayer,
    WMSLayer,
    GeoJSON,
    projections
)

## Data

For this demo we will use some NAIP data hosted on Azure.

https://azure.microsoft.com/fr-fr/services/open-datasets/catalog/naip/


The data is similar to the data hosted on [AWS](https://registry.opendata.aws/naip/), but using the one on Azure is easier because it offers direct `http` access without needing an AWS account.

In [3]:
# For this DEMO we will use this file
src_path = "https://naipblobs.blob.core.windows.net/naip/v002/al/2019/al_60cm_2019/30087/m_3008701_ne_16_060_20191115.tif"

## Tile Server

For this demo, we need to create a minimal tile server.

In [4]:
import json
from concurrent import futures

from tornado import web
from tornado import gen
from tornado.httpserver import HTTPServer
from tornado.concurrent import run_on_executor

from rio_tiler.io import COGReader
from rio_tiler.errors import TileOutsideBounds
from rio_tiler.profiles import img_profiles

class TileServer:
    
    def __init__(self, src_path):
        """Initialize Tornado app."""
        self.server = None
        self.app = web.Application([
            (r"^/tiles/(\w+)/(\d+)/(\d+)/(\d+)", TileHandler, {"url": src_path}),
        ])

    def start(self):
        """Start tile server."""
        self.server = HTTPServer(self.app)
        self.server.listen(8080)
  
    def stop(self):
        """Stop tile server."""
        if self.server:
            self.server.stop()


class TileHandler(web.RequestHandler):
    """Tile requests handler."""

    executor = futures.ThreadPoolExecutor(max_workers=16)

    def initialize(self, url):
        """Initialize tiles handler."""
        self.url = url

    @run_on_executor
    def _get_tile(self, tms, z, x, y):

        try:
            with COGReader(self.url, tms=morecantile.tms.get(tms)) as cog:
                img = cog.tile(x, y, z, indexes=(1,2,3))
        except TileOutsideBounds:
            raise web.HTTPError(404)

        prof = img_profiles.get("PNG", {})
        return img.render(img_format="PNG", **prof)

    @gen.coroutine
    def get(self, tms, z, x, y):
        """Retunrs tile data and header."""
        self.set_header("Access-Control-Allow-Origin", "*")
        self.set_header("Access-Control-Allow-Methods", "GET")
        self.set_header("Content-Type", "image/png")
        self.set_header("Cache-Control", "no-store, no-cache, must-revalidate")
        res = yield self._get_tile(tms, int(z), int(x), int(y))
        self.write(res)


ts = TileServer(src_path)
ts.start()

List the supported TMS from morecantile

In [5]:
print("Supported TMS:")
for name, tms in morecantile.tms.tms.items():
    print("-", name, "|",tms.crs)

Supported TMS:
- LINZAntarticaMapTilegrid | EPSG:5482
- EuropeanETRS89_LAEAQuad | EPSG:3035
- CanadianNAD83_LCC | EPSG:3978
- UPSArcticWGS84Quad | EPSG:5041
- NZTM2000 | EPSG:2193
- UTM31WGS84Quad | EPSG:32631
- UPSAntarcticWGS84Quad | EPSG:5042
- WorldMercatorWGS84Quad | EPSG:3395
- WorldCRS84Quad | EPSG:4326
- WebMercatorQuad | EPSG:3857


## WebMercator TMS

In [6]:
with COGReader(src_path, tms=morecantile.tms.get("WebMercatorQuad")) as cog:
    info = cog.spatial_info
print(info)

m = Map(center=(30.96, -87.90), zoom=info.minzoom, basemap={})

layer = TileLayer(
    url="http://127.0.0.1:8080/tiles/WebMercatorQuad/{z}/{x}/{y}",
    min_zoom=info.minzoom,
    max_zoom=info.maxzoom,
    opacity=1,
)
m.add_layer(layer)
m

bounds=(-87.94024818743995, 30.93522814625122, -87.87233769345906, 31.002283423611786) center=(-87.9062929404495, 30.968755784931503, 12) minzoom=12 maxzoom=18


Map(center=[30.96, -87.9], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

/Users/vincentsarago/Workspace/venv/py38/lib/python3.8/site-packages/rasterio/io.py:133: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  return writer(mempath, 'w+', driver=driver, width=width,


## WGS84 TMS

In [7]:
with COGReader(src_path, tms=morecantile.tms.get("WorldCRS84Quad")) as cog:
    info = cog.spatial_info
print(info)

m = Map(center=(30.96, -87.90), zoom=info.minzoom, basemap={}, crs=projections.EPSG4326)

layer = TileLayer(
    url="http://127.0.0.1:8080/tiles/WorldCRS84Quad/{z}/{x}/{y}",
    min_zoom=info.minzoom,
    max_zoom=info.maxzoom,
    opacity=1,
)
m.add_layer(layer)
m

bounds=(-87.94024818743995, 30.93522814625122, -87.87233769345906, 31.002283423611786) center=(-87.9062929404495, 30.968755784931503, 11) minzoom=11 maxzoom=17


Map(center=[30.96, -87.9], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [10]:
ts.stop()